In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from keras import layers, models, losses
from keras.layers import Dense, Dropout, concatenate
import cv2 as cv
import os
import librosa
import moviepy.editor as mp

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  print("gpu")
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
  except RuntimeError as e:
    print(e)

gpu


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pwd

/content


In [3]:
!pip install tf-models-official

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 9.1 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=50e73ce361f097789c6c7bc7ddaa11ad01341b99cdb09c4fbb06605b3262d8df
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [4]:
import os
os.chdir('/content/drive/MyDrive/Codes/')
!pwd

import importlib

import custom_file_movinet
# Reload the module to ensure the latest version is used
importlib.reload(custom_file_movinet)
from custom_file_movinet import *

/content/drive/MyDrive/Codes


In [5]:
video_model=get_vid_model()

Model: "movinet_classifier_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image (InputLayer)          [(None, None, None, Non   0         
                             e, 3)]                              
                                                                 
 movinet (Movinet)           ({'stem': (None, None,    911583    
                             None, None, 8),                     
                              'block0_layer0': (None             
                             , None, None, None, 8),             
                              'block1_layer0': (None             
                             , None, None, None, 32)             
                             , 'block1_layer1': (Non             
                             e, None, None, None, 32             
                             ),                                  
                              'block1_layer2':

In [6]:
aud_model=create_vggish_aud_model()


Load trained models

In [7]:
#from tensorflow.keras.applications import ResNet50

video_model.load_weights(r'/content/drive/MyDrive/Codes/Saved models/Saved Movinet/1_without_augmentation/models_new/final_vid_model_MoViNet_1.h5')
aud_model.load_weights(r"/content/drive/MyDrive/Codes/Saved models/Audio_model_multiclass_new.h5")

video_model.summary()
aud_model.summary()



Model: "movinet_classifier_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image (InputLayer)          [(None, None, None, Non   0         
                             e, 3)]                              
                                                                 
 movinet (Movinet)           ({'stem': (None, None,    911583    
                             None, None, 8),                     
                              'block0_layer0': (None             
                             , None, None, None, 8),             
                              'block1_layer0': (None             
                             , None, None, None, 32)             
                             , 'block1_layer1': (Non             
                             e, None, None, None, 32             
                             ),                                  
                              'block1_layer2':

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 8, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 32)                  │          20,608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 7)                   │             119 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,255 (83.03 KB)

 Trainable params: 21,255 (83.03 KB)

 Non-trainable params: 0 (0.00 B)

### Create audio-video combined function

In [8]:
#@title Helper functions

vggish_model = hub.load('https://tfhub.dev/google/vggish/1')

max_seconds = 8
max_frames = 10

def get_audio_data(path, max_seconds=10):
    audio_data, sample_rate = librosa.load(path, sr=16000)
    aud_len = audio_data.shape[0]
    max_samples = max_seconds * 16000

    if aud_len < max_samples:
        audio_data = np.pad(audio_data, (0, max_samples - aud_len), mode='constant')
    elif aud_len > max_samples:
        audio_data = audio_data[0:max_samples]

    frames = librosa.util.frame(audio_data, frame_length=16000, hop_length=16000)
    frames = np.asarray(frames.T).astype(np.float32)

    features = []
    for frame in frames:
        feature = vggish_model(frame)
        features.append(feature)
    features = tf.concat(features, axis=0)
    return features

def preprocess_video_frames(video_frames):
    processed_frames = []
    for frame in video_frames:
        frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        resized_frame = cv.resize(frame_rgb, (224, 224))/255.0
        processed_frames.append(resized_frame)
    return np.array(processed_frames)

def extract_video_frames(video_path, start_time, end_time, num_frames=max_frames):
    cap = cv.VideoCapture(video_path)
    fps = cap.get(cv.CAP_PROP_FPS)
    total_frames = int((end_time - start_time) * fps)
    frame_interval = total_frames // num_frames

    frames = []
    for i in range(num_frames):
        frame_number = start_time * fps + i * frame_interval
        cap.set(cv.CAP_PROP_POS_FRAMES, frame_number)
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
        else:
            break
    cap.release()
    return frames

def run_classification_on_clip(video_path, start_time, end_time):

    classes_list = ['Abuse', 'Riot', 'Car Accident', 'Nonviolence', 'Shooting', 'Explosion', 'Fighting']
    # Extract video frames
    video_frames = extract_video_frames(video_path, start_time, end_time)
    processed_video_frames = preprocess_video_frames(video_frames)

    # Extract audio features
    clip = mp.VideoFileClip(video_path).subclip(start_time, end_time)
    audio_path = 'temp_audio.wav'
    clip.audio.write_audiofile(audio_path, codec='pcm_s16le')
    audio_features = get_audio_data(audio_path,max_seconds)

    # Run predictions
    video_predictions = video_model.predict(np.expand_dims(processed_video_frames, axis=0))
    video_predictions = tf.keras.layers.Softmax()(video_predictions)
    audio_predictions = aud_model.predict(np.expand_dims(audio_features, axis=0))

    # Combine predictions
    print(f"video_predictions {video_predictions}")
    print(f"audio_predictions {audio_predictions}")
    print(f"video_predicted class {classes_list[np.argmax(video_predictions)]}")
    print(f"audio_predicted class {classes_list[np.argmax(audio_predictions)]}")
    combined_predictions = (video_predictions + audio_predictions) / 2
    final_class = np.argmax(combined_predictions)
    final_class = classes_list[final_class]

    # Clean up temporary audio file
    os.remove(audio_path)

    return final_class

# Process the entire movie in 10-second clips
def process_movie(video_path):
    classes_list = ['Abuse', 'Riot', 'Car Accident', 'Nonviolence', 'Shooting', 'Explosion', 'Fighting']
    # Get the total duration of the video
    cap = cv.VideoCapture(video_path)
    total_duration = int(cap.get(cv.CAP_PROP_FRAME_COUNT) / cap.get(cv.CAP_PROP_FPS))
    cap.release()

    clip_duration = max_seconds#10
    class_counts = {}
    total_clips = 0

    for start_time in range(0, total_duration, clip_duration):
        end_time = min(start_time + clip_duration, total_duration)
        if end_time - start_time < clip_duration:
            continue
        final_class = run_classification_on_clip(video_path, start_time, end_time)
        class_counts[final_class] = class_counts.get(final_class, 0) + 1
        total_clips += 1
        print(f"Clip {start_time}-{end_time} classified as class {final_class}")

    print("\nSummary of the whole video:")
    for cls, count in class_counts.items():
        percentage = (count / total_clips) * 100
        print(f"Class {cls}: {percentage:.2f}%")




# Creating dataset for final dense layer

In [16]:
from sklearn.utils import resample

if 'csv_path' not in vars():
    csv_path ="/content/drive/MyDrive/Codes/CSVs/1_First_method_vid_enhanced_for_multiple_classes"

loaded_train_df = pd.read_csv(os.path.join(csv_path,"train.csv"),usecols=['Name','tag'])
loaded_test_df = pd.read_csv(os.path.join(csv_path,"test.csv"),usecols=['Name','tag'])

vggish_model = hub.load('https://tfhub.dev/google/vggish/1')

max_seconds = 8
max_frames = 10

def get_audio_data(path, max_seconds=10):
    audio_data, sample_rate = librosa.load(path, sr=16000)
    aud_len = audio_data.shape[0]
    max_samples = max_seconds * 16000

    if aud_len < max_samples:
        audio_data = np.pad(audio_data, (0, max_samples - aud_len), mode='constant')
    elif aud_len > max_samples:
        audio_data = audio_data[0:max_samples]

    frames = librosa.util.frame(audio_data, frame_length=16000, hop_length=16000)
    frames = np.asarray(frames.T).astype(np.float32)

    features = []
    for frame in frames:
        feature = vggish_model(frame)
        features.append(feature)
    features = tf.concat(features, axis=0)
    return features

def preprocess_video_frames(video_frames):
    processed_frames = []
    for frame in video_frames:
        frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        resized_frame = cv.resize(frame_rgb, (224, 224))/255.0
        processed_frames.append(resized_frame)
    return np.array(processed_frames)

def extract_video_frames(video_path, start_time, end_time, num_frames=max_frames):
    cap = cv.VideoCapture(video_path)
    fps = cap.get(cv.CAP_PROP_FPS)
    total_frames = int((end_time - start_time) * fps)
    frame_interval = total_frames // num_frames

    frames = []
    for i in range(num_frames):
        frame_number = start_time * fps + i * frame_interval
        cap.set(cv.CAP_PROP_POS_FRAMES, frame_number)
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
        else:
            break
    cap.release()
    return frames

def get_aud_vid_predictions(video_path):

    # Get the total duration of the video
    cap = cv.VideoCapture(video_path)
    total_duration = int(cap.get(cv.CAP_PROP_FRAME_COUNT) / cap.get(cv.CAP_PROP_FPS))
    cap.release()

    clip_duration = max_seconds#10  # since our clips are short this will only capture the clip
    class_counts = {}
    total_clips = 0

    """
    for start_time in range(0, total_duration, clip_duration):
        end_time = min(start_time + clip_duration, total_duration)
    """

    start_time = 0
    end_time = min(start_time + clip_duration, total_duration)

    # Extract video frames
    video_frames = extract_video_frames(video_path, start_time, end_time)
    processed_video_frames = preprocess_video_frames(video_frames)

    # Extract audio features
    clip = mp.VideoFileClip(video_path).subclip(start_time, end_time)
    audio_path = 'temp_audio.wav'
    clip.audio.write_audiofile(audio_path, codec='pcm_s16le')
    audio_features = get_audio_data(audio_path,max_seconds)

    # Run predictions
    video_predictions = video_model.predict(np.expand_dims(processed_video_frames, axis=0))
    video_predictions = tf.keras.layers.Softmax()(video_predictions)
    audio_predictions = aud_model.predict(np.expand_dims(audio_features, axis=0))

    output_predictions = np.concatenate((np.squeeze(video_predictions),np.squeeze(audio_predictions)))

    # Clean up temporary audio file
    os.remove(audio_path)
    return output_predictions


def upsample(data_frame):
    val_counts =data_frame['tag'].value_counts()
    max_category_size = np.max(val_counts)
    upsampled_full_df=[]
    for category, subset_df in data_frame.groupby('tag'):
        if val_counts[category] < max_category_size:
            subset_df = resample(subset_df,
                                    replace=True,                    # Sample with replacement
                                    n_samples=max_category_size,     # Majority class size
                                    random_state=42)
        upsampled_full_df.append(subset_df)
    return pd.concat(upsampled_full_df).reset_index(drop=True)

upsampled_train_df = upsample(loaded_train_df)

def create_vid_dataset(df):
    classes_list = ['Abuse', 'Riot', 'Car Accident', 'Nonviolence', 'Shooting', 'Explosion', 'Fighting']

    label_list = []
    out_vec_list =[]
    for i in range(len(df)):
      path = df.iloc[i,0]
      label = df.iloc[i,1]

      try:
        out_vec = get_aud_vid_predictions(path)
        label_list.append(label)
        out_vec_list.append(out_vec)
      except Exception as exc:
        print(exc)
        continue
    data = {'Out_vec':out_vec_list,'tag':label_list}
    df = pd.DataFrame(data)
    upsampled_df = upsample(df)
    upsampled_df = upsampled_df.sample(frac=1).reset_index(drop=True)

    class_to_index = {class_name: index for index, class_name in enumerate(classes_list)}
    upsampled_df['tag'] = upsampled_df['tag'].map(class_to_index)
    return upsampled_df


In [ ]:
upsampled_train_df = create_vid_dataset(loaded_train_df)
upsampled_test_df = create_vid_dataset(loaded_test_df.sample(frac=0.4).reset_index(drop=True))

upsampled_train_df.head()

MoviePy - Writing audio in temp_audio.wav


MoviePy - Done.


1/1 [==============================] - 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
MoviePy - Writing audio in temp_audio.wav


MoviePy - Done.


1/1 [==============================] - 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
MoviePy - Writing audio in temp_audio.wav


...

1/1 [==============================] - 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
MoviePy - Writing audio in temp_audio.wav


MoviePy - Done.


1/1 [==============================] - 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
MoviePy - Writing audio in temp_audio.wav


MoviePy - Done.


1/1 [==============================] - 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
MoviePy - Writing audio in temp_audio.wav


In [15]:
upsampled_test_df.sample(frac=0.4).reset_index(drop=True)

,Out_vec,tag
0,"[0.7588998, 0.015963452, 0.023298396, 0.049056...",0
1,"[0.2794126, 0.0036989192, 0.008304048, 0.33102...",0
2,"[0.7771847, 0.0048660235, 0.0065176007, 0.0922...",0
3,"[0.063943006, 0.00013405674, 0.006283617, 0.84...",0


In [ ]:
final_dense_model = tf.keras.Sequential([
    layers.Input(shape=(14,)),
    layers.Dense(1,activation = 'linear')
])

num_epochs = 10

final_dense_model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.001),
                  loss = losses.SparseCategoricalCrossentropy(),
                  metrics=[keras.metrics.CategoricalAccuracy(), keras.metrics.Precision(), keras.metrics.Recall()])

callback = keras.callbacks.EarlyStopping(monitor='val_loss',patience=3)#3

# Convert DataFrames to NumPy arrays
X_train = np.array(upsampled_train_df['Out_vec'].tolist())  # Shape: (num_samples, 14)
y_train = np.array(upsampled_train_df['tag'].tolist())      # Shape: (num_samples,)

X_test = np.array(upsampled_test_df['Out_vec'].tolist())    # Shape: (num_samples, 14)
y_test = np.array(upsampled_test_df['tag'].tolist())        # Shape: (num_samples,)

history = final_dense_model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=8,
    epochs=num_epochs,
    callbacks=[callback],
    shuffle=True
)

In [ ]:
final_dense_model.save('/content/drive/MyDrive/Codes/Saved models/finale_dense_model.h5')

Evaluation

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

csv_path= "/content/drive/MyDrive/Codes/CSVs/1_First_method_vid_enhanced_for_multiple_classes"

if 'csv_path' not in vars():
    csv_path ="/content/drive/MyDrive/Codes/CSVs/1_First_method_vid_enhanced_for_multiple_classes"

loaded_train_df = pd.read_csv(os.path.join(csv_path,"train.csv"),usecols=['Name','tag'])

loaded_test_df = pd.read_csv(os.path.join(csv_path,"test.csv"),usecols=['Name','tag'])


def run_classification_on_clip_for_eval(video_path, start_time, end_time):

    classes_list = ['Abuse', 'Riot', 'Car Accident', 'Nonviolence', 'Shooting', 'Explosion', 'Fighting']
    # Extract video frames
    video_frames = extract_video_frames(video_path, start_time, end_time)
    processed_video_frames = preprocess_video_frames(video_frames)

    # Extract audio features
    clip = mp.VideoFileClip(video_path).subclip(start_time, end_time)
    audio_path = 'temp_audio.wav'
    clip.audio.write_audiofile(audio_path, codec='pcm_s16le')
    audio_features = get_audio_data(audio_path,max_seconds)

    # Run predictions
    video_predictions = video_model.predict(np.expand_dims(processed_video_frames, axis=0))
    video_predictions = tf.keras.layers.Softmax()(video_predictions)
    audio_predictions = aud_model.predict(np.expand_dims(audio_features, axis=0))

    # Combine predictions
    combined_predictions = (video_predictions + audio_predictions) / 2
    final_class = np.argmax(combined_predictions)
    final_class = classes_list[final_class]

    # Clean up temporary audio file
    os.remove(audio_path)

    return final_class

# Process the entire movie in 10-second clips
def pred_vid(video_path):
    classes_list = ['Abuse', 'Riot', 'Car Accident', 'Nonviolence', 'Shooting', 'Explosion', 'Fighting']
    # Get the total duration of the video
    cap = cv.VideoCapture(video_path)
    total_duration = int(cap.get(cv.CAP_PROP_FRAME_COUNT) / cap.get(cv.CAP_PROP_FPS))
    cap.release()

    clip_duration = max_seconds#10  # since our clips are short this will only capture the clip
    class_counts = {}
    total_clips = 0

    for start_time in range(0, total_duration, clip_duration):
        end_time = min(start_time + clip_duration, total_duration)
        final_class = run_classification_on_clip_for_eval(video_path, start_time, end_time)
    return final_class  # This will only work for clips around 10 seconds

def vid_eval(df):
    classes_list = ['Abuse', 'Riot', 'Car Accident', 'Nonviolence', 'Shooting', 'Explosion', 'Fighting']
    df= pd.get_dummies(df.loc[:,['Name','tag']], columns=['tag'])
    df.iloc[:,1:]= df.iloc[:,1:].astype(np.uint8)
    confusion_matrix = np.zeros((7,7))
    for i in range(len(loaded_test_df)):
      path = loaded_test_df.iloc[i,0]
      label = loaded_test_df.iloc[i,1]

      try:
        pred_class = pred_vid(path)
      except Exception as exc:
        print(exc)
        continue

      if pred_class is None:
        continue
      confusion_matrix[classes_list.index(label),classes_list.index(pred_class)]+=1
    accuracy = np.trace(confusion_matrix)/np.sum(confusion_matrix)
    print(f"accuracy {accuracy}")
    ax = sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=classes_list, yticklabels=classes_list)
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels')
    ax.set_title('Confusion Matrix')
    plt.show()
    return confusion_matrix




In [ ]:
for i in range(10):
  try:
    if i==5:
      1/0
      print(5)
    print(i)
  except  Exception as exc:
    print(exc)
    continue

0
1
2
3
4
division by zero
6
7
8
9


In [ ]:
vid_eval(loaded_test_df.sample(frac = 0.1))

MoviePy - Writing audio in temp_audio.wav


MoviePy - Done.


1/1 [==============================] - 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
MoviePy - Writing audio in temp_audio.wav


MoviePy - Done.


1/1 [==============================] - 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
MoviePy - Writing audio in temp_audio.wav


...

1/1 [==============================] - 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
MoviePy - Writing audio in temp_audio.wav


MoviePy - Done.


1/1 [==============================] - 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
MoviePy - Writing audio in temp_audio.wav


chunk:   0%|          | 0/177 [00:00<?, ?it/s, now=None]

OSError: Error in file /content/drive/MyDrive/Multi Class Dataset/Nonviolence/473.mp4, Accessing time t=10.08-10.12 seconds, with clip duration=10 seconds, 

In [ ]:
p=r"/content/drive/MyDrive/Multi Class Dataset/Abuse/40.mp4"
process_movie(p)

chunk:  43%|████▎     | 48/111 [18:57<00:00, 179.92it/s, now=None]

MoviePy - Writing audio in temp_audio.wav



chunk:  43%|████▎     | 48/111 [18:57<00:00, 179.92it/s, now=None]

MoviePy - Done.
1/1 [==============================] - 8s 8s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
video_predictions [[0.5232356  0.00487373 0.006957   0.11188627 0.23909117 0.02031887
  0.09363729]]
audio_predictions [[0.4451804  0.01180066 0.00227329 0.06760249 0.36594164 0.00432498
  0.10287654]]
video_predicted class Abuse
audio_predicted class Abuse
Clip 0-8 classified as class Abuse


chunk:  43%|████▎     | 48/111 [19:10<00:00, 179.92it/s, now=None]

MoviePy - Writing audio in temp_audio.wav



chunk:   0%|          | 0/111 [00:00<?, ?it/s, now=None]

OSError: Error in file /content/drive/MyDrive/Multi Class Dataset/Abuse/40.mp4, Accessing time t=10.12-10.16 seconds, with clip duration=10 seconds, 

In [ ]:
# Example usage
video_path = r'/content/drive/MyDrive/Multi Class Dataset/Fighting/1065.mp4'
process_movie(video_path)

MoviePy - Writing audio in temp_audio.wav


MoviePy - Done.


1/1 [==============================] - 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
video_predictions [[0.12020984 0.0053605  0.01900598 0.20392457 0.1134651  0.26439118
  0.27364287]]
audio_predictions [[0.06488045 0.00085516 0.00394835 0.7902981  0.03679961 0.00399699
  0.0992214 ]]
video_predicted class Fighting
audio_predicted class Nonviolence
Clip 0-8 classified as class Nonviolence

Summary of the whole video:
Class Nonviolence: 100.00%
